<a href="https://colab.research.google.com/github/brucecmd/tensorflow_in_action_code/blob/master/ExponentialMovingAverage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [20]:
v = tf.Variable(0, dtype=tf.float32) # 这里别忘记加dtype=，否则后面会报错
step = tf.Variable(0, trainable=False)
ema = tf.train.ExponentialMovingAverage(0.99, step)

maintain_averages_op = ema.apply([v])

with tf.Session() as sess:
  tf.global_variables_initializer().run()
  
  print(sess.run([v, ema.average(v)])) # 最开始的时候，值是一样的，都是0
  
  # 之后v变动一次之后，值就会变了，具体看公式：  min(0.99, 1 + step / 10 + step) = 0.1; 0.1 * 0 + 0.9 * 10 = 9 
  sess.run(tf.assign(v, 10))
  sess.run(maintain_averages_op)
  print(sess.run([v, ema.average(v)]))
  
  # 再变动v一次，还按上面的公式算; step没变，还是0；min后的值还是0.1，所以0.1*9 + 0.9*10=9.9 (0.1*shadow + 0.9*new_value)
  sess.run(tf.assign(v, 10))
  sess.run(maintain_averages_op)
  print(sess.run([v, ema.average(v)])) 
  
  # 再变动v一次，顺便把step变了; min(0.99， 1+10000/10/10000)=0.99, 0.99*shadow+0.01*new_value=0.99*9.9 + 0.01*10=9.901
  sess.run(tf.assign(v, 10))
  sess.run(tf.assign(step, 10000))
  sess.run(maintain_averages_op)
  print(sess.run([v, ema.average(v)]))

[0.0, 0.0]
[10.0, 9.0]
[10.0, 9.9]
[10.0, 9.901]
